# click으로 시도

In [29]:
%%time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time as t


options = webdriver.Chromeoptions()
options.add_argument('headless')
options.add_argument('window-size = 1920x1980')


driver = webdriver.Chrome('chromedriver.exe')

driver.get('https://everytime.kr')

driver.find_element_by_class_name('login').find_element_by_link_text('로그인').click()

driver.find_element_by_name('userid').send_keys('-------')
driver.find_element_by_name('password').send_keys('--------')

driver.find_element_by_class_name('submit').click()

driver.find_element_by_link_text('자유게시판').click()
driver.refresh()
url = []
time = []
title = []
content = []
vote = []
comment = []
n = 0
for page_number in range(1,501,1):
    page_url = driver.current_url
    driver.implicitly_wait(3)
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    driver.execute_script('window.scrollTo(0,0);')
    post_list = driver.find_elements_by_tag_name('article')
    
    for i in range(20):         
        
        
        post_url = post_list[i].find_element_by_tag_name('a').get_attribute('href')
        
        #post_list[i].click()
        #url.append(driver.current_url)
        url.append(post_url)
        time.append(driver.find_element_by_tag_name('time').get_attribute('title'))
        
        driver.get(post_url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html5lib')
        print('start')
        title.append(soup.select('h2.large')[0].text.strip())
        print(i)
        content.append(soup.select('p.large')[0].text.strip())
        vote.append(soup.select('li.vote')[0].text.strip())
        comment.append(soup.select('li.comment')[0].text.strip())
        driver.back()
        
    if page_number % 50 == 0:
        df = pd.DataFrame({'title' : title, 'content' : content, 'vote' : vote, 'comment' : comment, 'url' : url, 'time' : time})
        df = df[['title', 'content', 'comment', 'vote', 'time', 'url']]
        df.to_csv('everytime{}.csv'.format(page_number // 50))
        url = []
        time = []
        title = []
        content = []
        vote = []
        comment = []
    driver.implicitly_wait(20)
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    driver.find_element_by_css_selector('a.next').click()

start


IndexError: list index out of range

# BeautifulSoup 파싱 없이 바로
- driver에서 element로 바로 찾아서 append

In [100]:
%%time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time as t


options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size = 1920x1980')


driver = webdriver.Chrome('chromedriver.exe')

driver.get('https://everytime.kr')

driver.find_element_by_class_name('login').find_element_by_link_text('로그인').click()

driver.find_element_by_name('userid').send_keys('------')
driver.find_element_by_name('password').send_keys('--------')

driver.find_element_by_class_name('submit').click()

driver.find_element_by_link_text('자유게시판').click()
driver.refresh()

url = []
time = []
title = []
content = []
vote = []
comment = []
n = 0

for page_number in range(1,501,1):
    
    page_url = driver.current_url

    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    driver.execute_script('window.scrollTo(0,0);')
    t.sleep(1)
    post_url_list = [u.get_attribute('href') for u in driver.find_elements_by_css_selector('article a.article')]
    for i in range(2):
        #post_list = driver.find_elements_by_tag_name('article')
        #post_url = post_list[i].find_element_by_tag_name('a').get_attribute('href')
        post_url = post_url_list[i]
        #post_list[i].click()
        driver.get(post_url)
        
        t.sleep(1)
        url.append(post_url)
        time.append(driver.find_element_by_tag_name('time').get_attribute('title'))


        # method1
        title.append(driver.find_element_by_css_selector('article a.article h2.large').text)

        content.append(driver.find_element_by_css_selector('article a.article p.large').text.strip())
        vote.append(driver.find_element_by_css_selector('article a.article ul.status li.vote').text)
        comment.append(driver.find_element_by_css_selector('article a.article ul.status li.comment').text)

        driver.back()
        
        if page_number % 50 == 0:
            df = pd.DataFrame({'title' : title, 'content' : content, 'vote' : vote, 'comment' : comment, 'url' : url, 'time' : time})
            df = df[['title', 'content', 'comment', 'vote', 'time', 'url']]
            df.to_csv('everytime{}.csv'.format(page_number // 50))
            url = []
            time = []
            title = []
            content = []
            vote = []
            comment = []
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    t.sleep(1)
    driver.find_element_by_css_selector('a.next').click()

Wall time: 9.8 s
